# Generating Bash Code with Granite Code Models

> **NOTE:** This recipe assumes you are working on a Linux, MacOS, or other UNIX-compatible system. While we haven't tested on Windows, some of the examples may generate valid DOS or PowerShell output. See comments below.


## Setting Up

Ensure you are running python 3.10, 3.11, or 3.12 in a freshly-created virtual environment.

In [ ]:
import sys
assert sys.version_info >= (3, 10) and sys.version_info < (3, 13), "Use Python 3.10, 3.11, or 3.12 to run this notebook."

### Install dependencies

Granite Kitchen comes with a bundle of dependencies that are required for notebooks. See the list of packages in its [`setup.py`](https://github.com/ibm-granite-community/granite-kitchen/blob/main/setup.py). 

In [ ]:
! pip install "git+https://github.com/ibm-granite-community/utils.git" langchain_community langchain_ollama replicate

### Serving the Granite AI model


This notebook requires IBM Granite models to be served by an AI model runtime so that the models can be invoked or called. This notebook can use a locally accessible [Ollama](https://github.com/ollama/ollama) server to serve the models, or the [Replicate](https://replicate.com) cloud service.

During the pre-work, you may have either started a local Ollama server on your computer, or setup Replicate access and obtained an [API token](https://replicate.com/account/api-tokens).

## Select your model


Select a Granite Code model to use. Here we use a Langchain client to connect to the model. If there is a locally accessible Ollama server, we use an Ollama client to access the model. Otherwise, we use a Replicate client to access the model.

When using Replicate, if the `REPLICATE_API_TOKEN` environment variable is not set, or a `REPLICATE_API_TOKEN` Colab secret is not set, then the notebook will ask for your [Replicate API token](https://replicate.com/account/api-tokens) in a dialog box.

In [ ]:
import os
import requests
from langchain_ollama.llms import OllamaLLM
from langchain_community.llms import Replicate
from ibm_granite_community.notebook_utils import get_env_var

try: # Look for a locally accessible Ollama server for the model
    response = requests.get(os.getenv("OLLAMA_HOST", "http://127.0.0.1:11434"))
    model = OllamaLLM(model="granite-code:8b")
except Exception: # Use Replicate for the model
    model = Replicate(model="ibm-granite/granite-8b-code-instruct-128k",
                      replicate_api_token=get_env_var("REPLICATE_API_TOKEN"))


### Detect your operating system

We'll find it useful to determine the name of our operating system and use that string in queries. This is because shell commands sometimes have different options on Linux vs. MacOS, etc. We'll write our queries so they take this difference into account. We can use a helper function to determine your OS. Note that `platform.system()` returns `Windows` on Windows system.

> **TIPS:** If you are using MacOS, you can install Linux-compatible versions of many commands. Consider these two options:
> * Install GNU Coreutils on a Mac. See [these instructions](https://superuser.com/questions/476575/replace-os-xs-shell-commands-with-the-linux-versions).
> * Install [HomeBrew](https://brew.sh/) and use it to install Linux-compatible (and other) tools.

> **Note:** The [Google Colab](https://colab.research.google.com) runtime environment is Linux.

In [ ]:
import platform

def os_name():
    os_name = platform.system()
    # It turns out, using "MacOS" is better than "Darwin", which is what gets returned on MacOS.
    # For all other cases, the returned value should be fine as is, so we map the result to the desired
    # name, but only for MacOS...
    name_map = {'Darwin': 'MacOS'}
    shell_map = {'Windows': 'DOS'} # On Windows and use Power Shell, change from `DOS` to `Power Shell`.
    # ... then pass the os_name value as the second arg, which is used as the default return value.
    # For the shell name, return `bash` by default. (You can change this to zsh, fish, etc.)
    return name_map.get(os_name, os_name), shell_map.get(os_name, 'bash')

my_os, my_shell = os_name()
print(f"My OS is {my_os}. My shell is {my_shell}.")


## One-shot Prompting with Granite Code

In One-shot prompting, you provide the model with a question and no examples. The model will generate an answer given its training. Larger models tend to do better at this task.

Note how we add additional context to the user's input prompt, such as _"make sure you write code that works for _my_ system!"_ (We'll see another way to do this below.)

In [ ]:
from textwrap import dedent

prompt = dedent(f"""\
    Show me a {my_shell} script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time
    for each file in the output. Make sure you generate {my_shell} code that is {my_os}-compatible!"""
)

response = model.invoke(prompt)
print(response)

### Try the script

Remove any markdown formatting in the output and paste the commands generated into the next cell _**after the %%bash line shown**_. Also delete the `ls -l`, which is there to allow the cell to run without error if nothing is pasted there (e.g., in our CI/CD test system). So, for example, you might have something like the following:

```shell
%%bash
find dir -type d | do_something
...
```

The `%%bash` "magic" tells Jupyter to run the commands as a shell script instead of as Python code. You can omit lines like `#!/bin/bash` and keep or remove any comments `# this is a comment...`.

Does the script work? If not try running the query again. Also try modifying the query string. What difference do these steps make?

In [ ]:
%%bash
ls -l

## Few-shot Prompting with Granite Code

In few-shot prompting, you provide the model with a question and some examples. The model will generate an answer given its training. The additional examples help the model zero in on a pattern, which may be required for smaller models to perform well at this task.

### Provide a list of Q&A examples


One of the examples uses the `stat` command, which requires different syntax for Linux vs. MacOS systems.

> **NOTE:** If you are using a Windows system, try changing the "answers" in the `examples` cell to be valid Power Shell or DOS commands. You can ignore the `stat_flags` in the next cell.

In [ ]:
stat_flags = '-c "%y %n" {}'
if my_os == 'MacOS':
    stat_flags = '-f "%m %N" {}'
print(f"The 'stat' flags for my OS \'{my_os}\' and shell \'{my_shell}\' are \'{stat_flags}\'")

In [ ]:
examples = [
    {
        "question": "Recursively find files that match '*.js', and filter out files with 'excludeddir' in their paths.",
        "answer": "find . -name '*.js' | grep -v excludeddir",
    },
    {
        "question": "Dump \"a0b\" as hexadecimal bytes.",
        "answer": "printf \"a0b\" | od -tx1",
    },
    {
        "question": "Create a tar ball of all pdf files in the current folder and any subdirectories.",
        "answer": "find . -name '*.pdf' | xargs tar czvf pdf.tar",
    },
    {
        "question": "Sort all files and directories in the current directory, but no subdirectories, according to modification time, and print only the seven most recently modified items.",
        "answer": f"find . -maxdepth 1 -exec stat {stat_flags} ';' | sort -n -r | tail -n 7",
    },
    {
        "question": "Find all the empty directories in and under the current directory.",
        "answer": "find . -type d -empty",
    },
]

### Assemble the prompt template

Here we build up a chat prompt template from messages. See the [Langchain docs](https://python.langchain.com/docs/how_to/few_shot_examples_chat/#fixed-examples) for more details.

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{question}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

chat_template = ChatPromptTemplate.from_messages(
    [
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

print(chat_template.input_variables)


### View the completed prompt

Create a prompt and inspect the fully-interpolated chat template. Alternating Human/AI messages create a structure that the model will follow.

In [ ]:
# Construct a prompt with no tabs and newlines.
prompt = dedent(f"""\
    Show me a {my_shell} script to print the first 50 files found under the current working directory
    that have been modified within the last week. Make sure you show the last modification time
    for each file in the output."""
).replace("\n", "")

print(chat_template.format(question=prompt))

### Run the model

In [ ]:
chain = chat_template | model
response = chain.invoke({"question": prompt})
print(response)

## Adding a System Prompt

Finally, a _system prompt_ is the preferred way to provide additional instructions and clarity about the context for a task, especially when this same information applies for _all_ user queries in the application. When you are building an AI-enabled application for a set of use cases, you will probably spend a lot of time refining the system prompt to maximize the quality of the results!

Here we define a system prompt to let the model know what we expect from it, including the expected language and os compatibility. To separate the system prompt from the user prompt, we use a chat template, which separates the prompt into messages and allows us to indicate the role of the speaker.


In [ ]:
from langchain_core.messages import SystemMessage

system_prompt = SystemMessage(content=dedent(f"""\
    You are a helpful software engineer. You write clear, concise, well-commented code. Do not output anything but the code.
    Make sure you only generate {my_shell} code that is {my_os}-compatible!"""
))

chat_template_with_system_message = ChatPromptTemplate.from_messages(
    [
        system_prompt,
        few_shot_prompt,
        ("human", "{question}"),
    ]
)

print(chat_template_with_system_message.input_variables)

### Run the model

Note that the model has removed its chatter from around the code block.

In [ ]:
chain_sys = chat_template_with_system_message | model
response = chain_sys.invoke({"question": prompt})
print(response)

### For further study
- Try different queries. Test them to see if the model generated them correctly.
- Try invoking the model chain several times. How do the responses change from one invocation to the next?
- Try adding more examples to the `examples` string or modifying the ones shown. Does this affect the outputs?